# spot and margin

In [3]:

import requests
import pandas as pd
url = "https://api.coinex.com/v1/market/info"

payload={}
files={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, files=files)    
response=response.json()  
response=response['data']
response=pd.DataFrame(response)
response=response.T
response.drop(response.loc[response['pricing_name']!='USDT'].index, inplace=True)
response

,name,min_amount,maker_fee_rate,taker_fee_rate,pricing_name,pricing_decimal,trading_name,trading_decimal
XNOUSDT,XNOUSDT,1,0.003,0.003,USDT,4,XNO,8
PHAUSDT,PHAUSDT,5,0.003,0.003,USDT,4,PHA,8
POLISUSDT,POLISUSDT,1,0.003,0.003,USDT,4,POLIS,8
PSPUSDT,PSPUSDT,50,0.003,0.003,USDT,4,PSP,8
KAVAUSDT,KAVAUSDT,0.5,0.003,0.003,USDT,4,KAVA,8
...,...,...,...,...,...,...,...,...
SRKUSDT,SRKUSDT,1000,0.003,0.003,USDT,6,SRK,8
SUNUSDT,SUNUSDT,100,0.003,0.003,USDT,6,SUN,8
BANUSDT,BANUSDT,500,0.003,0.003,USDT,6,BAN,8
VOXELUSDT,VOXELUSDT,1,0.003,0.003,USDT,4,VOXEL,8


# margin

In [36]:
import requests
import pandas as pd
url = "https://api.coinex.com/v1/margin/market"

payload={}
files={}
headers = {}

margin = requests.request("GET", url, headers=headers, data=payload, files=files)
margin=margin.json()
margin=margin['data']
margin=pd.DataFrame([margin.keys()]).T
margin = margin.rename({0: 'margin'}, axis=1)
i=1
while i< len(margin):
    
    t=margin.loc[i,'margin']
    usdt=t[-4]+t[-3]+t[-2]+t[-1]
    if usdt!='USDT':
#         print(i)
#         print(margin.loc[i,'margin'])
        margin.drop(i,inplace=True)
        margin.reset_index(drop=True,inplace=True)
        i=i-1
    i=i+1
for i in range(len(margin)):
    t=margin.loc[i,'margin']
    t=t.split('USDT')[0]
    margin.loc[i,'margin']=t+'/'+'USDT'
print(margin)

        margin
0     BTC/USDT
1     ETH/USDT
2     BCH/USDT
3     LTC/USDT
4     BSV/USDT
..         ...
106  REEF/USDT
107  CTSI/USDT
108   INJ/USDT
109   RLC/USDT
110   XNO/USDT

[111 rows x 1 columns]


In [78]:
margin

,margin
0,BTC/USDT
1,ETH/USDT
2,BCH/USDT
3,LTC/USDT
4,BSV/USDT
...,...
106,REEF/USDT
107,CTSI/USDT
108,INJ/USDT
109,RLC/USDT


# future

In [46]:
import requests
import pandas as pd
url = "https://api.coinex.com/perpetual/v1/market/list"

payload={}
files={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, files=files)    
response=response.json()  
response=response['data']
response=pd.DataFrame(response)
response.drop(response.loc[response['money']!='USDT'].index, inplace=True)
symbol=response['stock']+'/'+response['money']
symbol

1       BTC/USDT
2       BCH/USDT
4       ETH/USDT
5     1INCH/USDT
6      AAVE/USDT
7       ADA/USDT
8      ALGO/USDT
9      ANKR/USDT
10       AR/USDT
11     ATOM/USDT
12     AVAX/USDT
13      AXS/USDT
14      BAT/USDT
15      BNB/USDT
16      BSV/USDT
17     CAKE/USDT
18     CELO/USDT
19      CHZ/USDT
20     COMP/USDT
21      CRO/USDT
22      CRV/USDT
23     DASH/USDT
24      DGB/USDT
25     DOGE/USDT
26      DOT/USDT
27     DYDX/USDT
28     EGLD/USDT
29      ENJ/USDT
30      ENS/USDT
31      EOS/USDT
32      ETC/USDT
33      FIL/USDT
34     FLOW/USDT
35      FTM/USDT
36      FTT/USDT
37     GALA/USDT
38      GRT/USDT
39     HBAR/USDT
40      HNT/USDT
41      HOT/USDT
42      ICP/USDT
43      ICX/USDT
44     IOST/USDT
45     IOTA/USDT
46     IOTX/USDT
47     KAVA/USDT
48     KLAY/USDT
49      KSM/USDT
50     LINK/USDT
51      LPT/USDT
52      LRC/USDT
53      LTC/USDT
54     LUNA/USDT
55     MANA/USDT
56    MATIC/USDT
57     MINA/USDT
58      MKR/USDT
59     NEAR/USDT
60      NEO/US

In [35]:
import requests
import pandas as pd
import datetime

def coinex_future(market,period,limit):
        headers = {
                'Content-Type': 'application/json; charset=utf-8',
                'Accept': 'application/json',    }
        result = requests.get(
            'https://api.coinex.com/perpetual/v1/market/kline?market={market}&type={period}&limit={limit}'.format(
                market=market,period=period,limit=limit
            ),
            headers=headers
        )

        df = result.json()
        df=pd.DataFrame(df["data"])
        df.columns = ['Timestamp', 'open', 'close', 'high', 'low', 'volume', 'value']
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
        df['date'] = df['Timestamp'].dt.strftime("%d/%m/%Y")
        df['time-utc'] = df['Timestamp'].dt.strftime("%H:%M:%S")
        df.drop(['value','Timestamp'], inplace=True, axis=1)
        df=df[["date", "time-utc", "open", "high", "low", "close", "volume"]]
        return df

market='BTCUSDT'
period='1hour'
limit='100'
df=coinex_future(market,period,limit)
df

,date,time-utc,open,high,low,close,volume
0,16/03/2022,07:00:00,39628.11,39706.04,39406.32,39619.03,229.5020
1,16/03/2022,08:00:00,39619.99,40487.68,39600.51,40278.76,618.5855
2,16/03/2022,09:00:00,40303.40,40699.99,40220.01,40483.70,301.4082
3,16/03/2022,10:00:00,40483.71,40643.30,40144.76,40296.31,245.6835
4,16/03/2022,11:00:00,40291.78,40655.62,40213.71,40488.70,268.3777
...,...,...,...,...,...,...,...
95,20/03/2022,06:00:00,41898.85,42000.00,41850.00,41901.00,48.8017
96,20/03/2022,07:00:00,41901.00,41966.81,41789.14,41861.14,79.2507
97,20/03/2022,08:00:00,41864.65,41931.82,41827.10,41868.54,67.0117
98,20/03/2022,09:00:00,41868.53,41874.73,41589.34,41780.04,192.9931


In [7]:
from datetime import timedelta, date, datetime
import time
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import ccxt 


#--------------------------------------------
timeframe='30m'
limit=400

counter=0
msec = 1000
hold = 30




timeframes = (['1m',1,'minutes',limit*1,60*msec],['3m',3,'minutes',limit*3,60*msec],['5m',5,'minutes',limit*5,60*msec],['15m',15,'minutes',limit*15,60*msec],['30m',30,'minutes',limit*30,60*msec],['1h',1,'hours',limit*1,60*60*msec],['2h',2,'hours',limit*2,60*60*msec],['4h',4,'hours',limit*4,60*60*msec],['6h',6,'hours',limit*6,60*60*msec],['8h',8,'hours',limit*8,60*60*msec],['12h',12,'hours',limit*12,60*60*msec],['1d',1,'days',limit*1,60*60*24*msec],['3d',3,'days',limit*3,60*60*24*msec],['1w',1,'weeks',limit*7,60*60*24*7*msec])
timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit' , 'period'])
timeframes.set_index('timeframe', inplace=True)

timeframes['limit']  = timeframes['limit'].astype(np.float64)
timeframes['period'] = timeframes['period'].astype(np.float64)


exchange = ccxt.coinex()
def run():
    now = exchange.milliseconds()
    if timeframes.loc[timeframe, 'name_time'] == 'minutes':
        from_datetime=pd.to_datetime(now, unit='ms') - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

    if timeframes.loc[timeframe, 'name_time'] == 'hours':
        from_datetime=pd.to_datetime(now, unit='ms') - timedelta(hours=timeframes.loc[timeframe, 'limit'])

    if timeframes.loc[timeframe, 'name_time'] == 'days':
        from_datetime=pd.to_datetime(now, unit='ms') - timedelta(days=timeframes.loc[timeframe, 'limit'])



    from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S%z')
    from_timestamp = exchange.parse8601(from_datetime)
    data = []

    while from_timestamp < now:
        #writer = ExcelWriter('1min.xlsx')
        try:
            ohlcvs = exchange.fetch_ohlcv('BTC/USDT', timeframe, from_timestamp)
            #print(candles, from_timestamp)
            print( 'Fetched', len(ohlcvs), 'candles')
            if len(ohlcvs) > 0:
                first = ohlcvs[0][0]
                last = ohlcvs[-1][0]

                from_timestamp = ohlcvs[-1][0] + timeframes.loc[timeframe, 'period'] * timeframes.loc[timeframe, 'time']
                data += ohlcvs

            df = pd.DataFrame(data, columns=['Timestamp','open','high','low','close', 'volume'])
            df['Timestamp'] = pd.DataFrame(df['Timestamp'].apply(exchange.iso8601))
            #save_excel = df.to_excel(writer, sheet_name='2017_CURRENT')
            #writer.save()

            print('first time:',pd.to_datetime(first, unit='ms'),'\nlast time:',pd.to_datetime(first, unit='ms'))
        except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:

            print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
            time.sleep(hold)
    return df
df=run()
df

Fetched 100 candles
first time: 2022-03-13 14:00:00 
last time: 2022-03-13 14:00:00


,Timestamp,open,high,low,close,volume
0,2022-03-13T14:00:00.000Z,38740.67,38921.93,38729.02,38888.01,3.879408
1,2022-03-13T14:30:00.000Z,38888.01,38901.14,38818.28,38839.42,1.226297
2,2022-03-13T15:00:00.000Z,38853.30,39076.60,38853.30,38959.47,8.596285
3,2022-03-13T15:30:00.000Z,38962.36,39010.07,38917.74,38939.00,2.231244
4,2022-03-13T16:00:00.000Z,38939.00,38997.45,38855.68,38944.99,2.802909
...,...,...,...,...,...,...
95,2022-03-15T13:30:00.000Z,38791.22,38859.69,38484.86,38709.81,8.875716
96,2022-03-15T14:00:00.000Z,38708.53,39057.98,38667.09,39033.51,22.525671
97,2022-03-15T14:30:00.000Z,39033.64,39185.41,38912.06,39037.53,11.782773
98,2022-03-15T15:00:00.000Z,39033.61,39170.00,38937.71,39144.41,18.053163


CCXT HAVE JUST MARGIN